# Puzzle 6

“Why yes, I did have that rug for a little while in my living room! My cats can’t see a thing but they sure chased after the squirrel on it like it was dancing in front of their noses.

“It was a nice rug and they were surely going to ruin it, so I gave it to my cousin, who was moving into a new place that had wood floors.

“She refused to buy a new rug for herself–she said they were way too expensive. She’s always been very frugal, and she clips every coupon and shops every sale at Noah’s Market. In fact I like to tease her that Noah actually loses money whenever she comes in the store.

“I think she’s been taking it too far lately though. Once the subway fare increased, she stopped coming to visit me. And she’s really slow to respond to my texts. I hope she remembers to invite me to the family reunion next year.”

Can you find her cousin’s phone number?

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import os
from nameparser import HumanName
import datetime as dt

In [2]:
dir = "/Users/jattenberg/Downloads/noahs-csv/"
files = os.listdir(dir)
names = [n.split(".")[0] for n in files]
dataframe_arr = [pd.read_csv(dir+f) for f in files]

dfs = dict(zip(names, dataframe_arr))

In [14]:
products = dfs["noahs-products"]
order_items = dfs["noahs-orders_items"]
orders = dfs["noahs-orders"]
customers = dfs["noahs-customers"]

person_from_last_time = customers.set_index("customerid").loc[7675]
last_name = HumanName(person_from_last_time["name"]).last

### Idea 1: Family reunion, who has the same last name?  

In [16]:
customers[customers.name.apply(lambda x: HumanName(x).last == last_name)]

,customerid,name,address,citystatezip,birthdate,phone
391,1392,Candice Koch,3817 W Lee Rd,"Richmond, VA 23173",1992-12-15,948-616-1092
1758,2759,James Koch,18 Cyrus Ave,"Brooklyn, NY 11229",1975-12-18,716-813-0669
4502,5503,Denise Koch,107B Hoyt St,"Brooklyn, NY 11217",1969-03-30,585-589-0502
6585,7586,Kyle Koch,539 Vincent Ave,"Bronx, NY 10465",1959-12-11,516-648-7363
6674,7675,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411


### None of these
### Idea 2: is there something in the price around being frugal? 

In [25]:
order_products = order_items.merge(products, on="sku")
discount_order_products = order_products[order_products.unit_price < order_products.wholesale_cost]
discount_orders_with_products = discount_order_products.merge(orders, on="orderid")

discount_orders_with_products


,orderid,sku,qty,unit_price,desc,wholesale_cost,customerid,ordered,shipped,items,total
0,94023,COL0820,1,19.73,Noah's Lunchbox (blue),20.96,7121,2019-08-18 12:18:05,2019-08-18 18:30:00,NaN,19.73
1,61840,TOY8907,1,9.94,Automatic Train,11.39,10255,2018-10-01 21:16:45,2018-10-04 19:30:00,NaN,13.94
2,158593,TOY8907,1,11.25,Automatic Train,11.39,10813,2021-05-26 01:40:34,2021-05-26 01:40:34,NaN,43.72
3,167507,KIT5813,1,6.32,Stone Tongs,6.37,8677,2021-08-23 09:42:36,2021-08-23 09:42:36,NaN,23.62
4,10517,KIT3981,1,5.34,Strainer,5.51,7883,2017-05-07 03:00:17,2017-05-07 16:45:00,NaN,5.34
...,...,...,...,...,...,...,...,...,...,...,...
2561,190255,PET8075,1,5.54,Rabbit Food,5.65,8612,2022-04-10 21:04:36,2022-04-15 12:45:00,NaN,28.53
2562,65260,KIT7336,1,1.90,Power Eggplant Scoop,1.92,8429,2018-11-04 12:27:40,2018-11-04 12:27:40,NaN,10.12
2563,66744,KIT7336,1,1.88,Power Eggplant Scoop,1.92,4795,2018-11-19 22:40:47,2018-11-23 08:00:00,NaN,26.39
2564,195331,KIT7336,1,1.86,Power Eggplant Scoop,1.92,1689,2022-05-30 16:44:16,2022-05-30 19:30:00,NaN,1.86


In [30]:
person = (
    discount_orders_with_products.drop_duplicates("orderid")
        .groupby("customerid")
        .orderid
        .count()
        .to_frame()
        .reset_index()
        .rename({"orderid": "sales"}, axis=1)
        .sort_values("sales", ascending=False)
        .head(1)
        .merge(customers, on="customerid")
)
person

,customerid,sales,name,address,citystatezip,birthdate,phone
0,8342,15,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316
